In [14]:
import pandas as pd
import psycopg2
from tqdm import tqdm
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer
import pandas as pd

In [15]:
# # Paramètres de connexion à la base de données
# dbname = 'postgres'
# user = 'postgres'
# password = '0000'
# host = 'localhost'  # ou l'adresse IP de votre serveur PostgreSQL
# port = '5432'       # le port par défaut pour PostgreSQL est 543

In [16]:
# Informations d'identification PostgreSQL
user = "terence"
password = "Canon0-Spinout8-Debatable6-Proofread2"
host = "stockage-donnee-foncieres-gr02.postgres.database.azure.com"
port = 5432
dbname = "postgres"

In [17]:
# Chaîne de connexion
conn_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"

In [18]:
# Définir la chaîne de connexion SQLAlchemy
engine = create_engine(conn_string)

In [19]:
# Connexion à la base de données
conn = engine.connect()


In [20]:
# Chargez le fichier CSV dans un DataFrame Pandas
df = pd.read_csv('./src/datasets/csv/valeurs_foncieres_affichage.csv', encoding='utf-8', dtype={'code_commune': str, 'code_departement': str})

In [21]:
# Supprimer la colonne inutile "Unnamed: 0"
df = df.drop(columns=['Unnamed: 0'])

In [22]:
# Nom de la table dans la base de données
table_name = 'valeur_fonciere'

In [23]:
# Créer la table en fonction de l'en-tête du fichier CSV
metadata = MetaData()
columns = [
    Column('date_mutation', String),
    Column('valeur_fonciere', Float),
    Column('code_commune', String),
    Column('code_departement', String),
    Column('type_local', String),
    Column('surface_reelle_bati', Float),
    Column('nombre_pieces_principales', Integer),
    Column('surface_terrain', Float)
]
table = Table(table_name, metadata, *columns)
metadata.create_all(engine)
print("La table a été créée avec succès.")

La table a été créée avec succès.


In [25]:
# Nombre total de lignes dans le DataFrame
total_rows = len(df)

# Itérer sur les lignes du DataFrame et insérer les données dans la base de données
with tqdm(total=total_rows, desc="Insertion des données dans la base de données") as pbar:
    for index, row in df.iterrows():
        # Exemple d'une requête d'insertion de données
        insert_query = f"""
        INSERT INTO {table_name} (date_mutation, valeur_fonciere, code_commune, code_departement, type_local, surface_reelle_bati, nombre_pieces_principales, surface_terrain)
        VALUES ('{row['date_mutation']}', {row['valeur_fonciere']}, '{row['code_commune']}', '{row['code_departement']}', '{row['type_local']}', {row['surface_reelle_bati']}, {row['nombre_pieces_principales']}, {row['surface_terrain']});
        """
        conn.execute(insert_query)
        
        # Mettre à jour la barre de progression
        pbar.update(1)

# Commit des modifications
conn.commit()

# Fermeture du curseur et de la connexion à la base de données
conn.close()

Insertion des données dans la base de données:   0%|          | 0/4046379 [00:00<?, ?it/s]


ArgumentError: List argument must consist only of tuples or dictionaries